Exploration Fichiers CSV


In [1]:
import pandas as pd
import gdown

In [2]:
# Liens vers les fichiers de données IMDb
# Ces fichiers sont disponibles en téléchargement sur le site IMDb
link_title_basics = "https://datasets.imdbws.com/title.basics.tsv.gz"
link_title_akas = "https://datasets.imdbws.com/title.akas.tsv.gz"
link_title_crew = "https://datasets.imdbws.com/title.crew.tsv.gz"
#link_title_episode = "https://datasets.imdbws.com/title.episode.tsv.gz" -> non utilisé car pas de séries
link_title_principals = "https://datasets.imdbws.com/title.principals.tsv.gz"
link_title_ratings = "https://datasets.imdbws.com/title.ratings.tsv.gz"
link_name_basics = "https://datasets.imdbws.com/name.basics.tsv.gz"
link_tmdb = "https://drive.google.com/file/d/1VB5_gl1fnyBDzcIOXZ5vUSbCY68VZN1v/view"

In [ ]:
# Fichier TMDB
file_id = "1VB5_gl1fnyBDzcIOXZ5vUSbCY68VZN1v"
output = "../ressources/tmdb_movies.csv"
#gdown.download(id=file_id,
#               output=output,
#               quiet=False,      # affiche la barre de progression
#               fuzzy=True)  

In [4]:
df_name_basics = pd.read_csv(link_name_basics, sep="\t", low_memory=False)
df_title_principals = pd.read_csv(link_title_principals, sep="\t", low_memory=False)
df_title_crew = pd.read_csv(link_title_crew, sep="\t", low_memory=False)
df_title_akas = pd.read_csv(link_title_akas, sep="\t", low_memory=False)
df_title_ratings = pd.read_csv(link_title_ratings, sep="\t", low_memory=False)
df_tmdb = pd.read_csv(output, low_memory=False)
df_title_basics = pd.read_csv(link_title_basics, sep="\t", low_memory=False)


In [5]:
# Début du traitement des fichiers IMDb
# Début nettoyage title.basics.tsv.gz
# Tous les genres de films existants dans le fichier title.basics.tsv.gz
genre_exploded = (
    df_title_basics["genres"]
    .str.split(",")
    .explode()
    .str.strip()
)
genre_exploded.unique()

array(['Documentary', 'Short', 'Animation', 'Comedy', 'Romance', 'Sport',
       'News', 'Drama', 'Fantasy', 'Horror', 'Biography', 'Music', 'War',
       'Crime', 'Western', 'Family', 'Adventure', 'Action', 'History',
       'Mystery', '\\N', 'Sci-Fi', 'Musical', 'Thriller', 'Film-Noir',
       'Talk-Show', 'Game-Show', 'Reality-TV', 'Adult', nan], dtype=object)

In [6]:
# Copy
df_title_basics_copy = df_title_basics.copy()
df_title_basics_copy.tail(3)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
11665847,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,Drama
11665848,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short
11665849,tt9916880,tvEpisode,Horrid Henry Knows It All,Horrid Henry Knows It All,0,2014,\N,10,"Adventure,Animation,Comedy"


In [7]:
# Remplacer NaN et \\N par "Unknown"
df_title_basics_copy["genres"] = df_title_basics_copy["genres"].str.replace("\\N", "Unknown", regex=False)
df_title_basics_copy["genres"] = df_title_basics_copy["genres"].fillna("Unknown")

# Genre non gardé : Documentary / Short / News / Talk-show / Game-Show / Reality-TV / Adult 
# On les enlève de la liste des genres
list_genre_to_drop = ["Documentary", "Short", "News", "Talk-show", "Game-Show", "Reality-TV", "Adult"]
regex_pattern = '|'.join(map(lambda x: f'({x})', list_genre_to_drop))
df_title_basics_copy = df_title_basics_copy[~df_title_basics_copy['genres'].str.contains(regex_pattern, regex=True)]

display(df_title_basics_copy.head(3))

C:\Users\User\AppData\Local\Temp\ipykernel_32760\2010509105.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_title_basics_copy = df_title_basics_copy[~df_title_basics_copy['genres'].str.contains(regex_pattern, regex=True)]


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
208,tt0000211,short,"The Astronomer's Dream; or, The Man in the Moon",La lune à un mètre,0,1898,\N,3,"Comedy,Fantasy,Horror"


In [8]:
# Verifier les différents types de films
df_title_basics_copy['titleType'].unique()

# On récupere uniquement les films
df_title_basics_copy = df_title_basics_copy[df_title_basics_copy['titleType'] == 'movie']

# On enlève les films pour adultes

df_title_basics_copy = df_title_basics_copy[df_title_basics_copy['isAdult'] == '0']

# On enlève les films sans titre
df_title_basics_copy['primaryTitle'] = df_title_basics_copy['primaryTitle'].fillna('None.')

display(df_title_basics_copy[df_title_basics_copy['primaryTitle'].isna() == False].head(3))




,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
498,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,Unknown
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography"


In [9]:
# Jointure entre df_title_basics_copy et df_tmdb, clé communune : tconst et imdb_id
df_merged = pd.merge(df_title_basics_copy, df_tmdb, left_on='tconst', right_on='imdb_id', how='left')

# On enlève les colonnes inutiles
df_merged = df_merged.drop(columns=['isAdult', 'originalTitle', 'endYear', 'adult', 'budget', 'homepage', 'imdb_id', 'original_title', 'titleType', 'runtime'])

# On transforme les colonnes 'runtimeMinutes', 'id', 'popularity', 'runtime', 'revenue', 'vote_average', 'vote_count' en numérique
df_merged['runtimeMinutes'] = pd.to_numeric(df_merged['runtimeMinutes'], errors='coerce', downcast='signed').astype('Int64')  
df_merged['id'] = pd.to_numeric(df_merged['id'], errors='coerce', downcast='integer').astype('Int64')  
df_merged['popularity'] = pd.to_numeric(df_merged['popularity'], errors='coerce', downcast='signed')
df_merged['revenue'] = pd.to_numeric(df_merged['revenue'], errors='coerce', downcast='signed')
df_merged['vote_average'] = pd.to_numeric(df_merged['vote_average'], errors='coerce', downcast='signed')
df_merged['vote_count'] = pd.to_numeric(df_merged['vote_count'], errors='coerce', downcast='signed').astype('Int64')

# On transforme la colonne 'startYear' en datetime
df_title_basics_copy['startYear'] = pd.to_numeric(df_title_basics_copy['startYear'], errors='coerce',downcast ='signed').astype('Int64')
df_merged['startYear'] = pd.to_datetime(df_merged['startYear'], format='%Y', errors='coerce').dt.to_period('Y')

# On transforme la colonne 'release_date' en datetime
df_merged['release_date'] = pd.to_datetime(df_merged['release_date'], format='%Y-%m-%d', errors='coerce').dt.date

# On filtre la colonne 'production_country' pour ne garder que les pays de production souhaités : USA,FR,GB
df_filtered = df_merged[df_merged['production_countries'].str.contains('USA|GB|FR', na=False)]

# On réorganise les colonnes : 'tconst', 'primaryTitle', 'title', 'startYear', 'release_date', 'genre_x', 'genre_y', 'production_countries', 'runtimeMinutes', 'vote_average', 'vote_count', 'popularity', 'revenue', 'tagline', 'overview', 'id'
df_filtered = df_filtered[['tconst', 'primaryTitle', 'title', 'startYear', 'release_date', 'genres_x', 'genres_y', 'production_countries', 'runtimeMinutes', 'vote_average', 'vote_count', 'popularity', 'revenue', 'tagline', 'overview', 'id']]

# Renomme la colonne 'genres_x' en 'genres_df_title_basics' et genres_y en 'genres_df_tmdb'
df_filtered = df_filtered.rename(columns={'genres_x': 'genres_df_title_basics', 'genres_y': 'genres_df_tmdb'})

# On enlève les doublons de la colonne 'primaryTitle'
df_filtered = df_filtered.drop_duplicates(subset=['primaryTitle'])

df_filtered

,tconst,primaryTitle,title,startYear,release_date,genres_df_title_basics,genres_df_tmdb,production_countries,runtimeMinutes,vote_average,vote_count,popularity,revenue,tagline,overview,id
3,tt0000591,The Prodigal Son,The Prodigal Son,1907,1907-06-19,Drama,['Drama'],['FR'],90,0.000,0,0.600,0.0,NaN,The first feature-length motion picture produc...,396922
89,tt0001790,"Les Misérables, Part 1: Jean Valjean",Les Misérables - Part 1: Jean Valjean,1913,1913-01-01,Drama,['History'],['FR'],60,1.000,1,0.874,0.0,NaN,The story begins with Jean Valjean as a humble...,282219
143,tt0002375,La mort du duc d'Enghien,The Death of the Duke of Enghien,1912,1912-01-02,Unknown,"['Drama', 'History']",['FR'],<NA>,0.000,1,0.823,0.0,NaN,Directed by Albert Capellani.,285335
146,tt0002406,Oliver Twist,Oliver Twist,1912,1912-10-24,Drama,[],['GB'],<NA>,1.000,1,0.621,0.0,NaN,An orphan named Oliver Twist meets a pickpocke...,465493
157,tt0002461,The Life and Death of King Richard III,The Life and Death of King Richard III,1912,1912-10-15,Drama,"['Drama', 'History', 'Adventure']","['FR', 'US']",55,4.700,7,1.051,0.0,NaN,Shakespeare's tragedy of the wicked and hump-b...,46758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565305,tt9904802,Enemy Lines,Enemy Lines,2020,2020-04-24,"Action,Drama,War","['War', 'Drama']",['GB'],92,5.400,28,5.523,0.0,Courage has no border.,"In the frozen, war torn landscape of occupied ...",679796
565331,tt9908390,Le lion,The Lion,2020,2020-01-29,Comedy,['Comedy'],['FR'],95,5.328,172,7.731,0.0,NaN,A psychiatric hospital patient pretends to be ...,589970
565335,tt9908636,Das Spiel der Hoffnung,Das Spiel der Hoffnung,2017,2017-05-25,Fantasy,['Fantasy'],['GB'],100,0.000,0,0.897,0.0,NaN,Follows the central character from infant inno...,688906
565455,tt9916190,Safeguard,Safeguard,2020,2020-09-07,"Action,Adventure,Thriller","['Action', 'Thriller', 'Crime']",['GB'],95,6.500,2,6.124,0.0,Protecting your family comes at a cost.,When a Japanese restaurant is extorted by the ...,633490


In [10]:
# Création d'un df pour les titres de films en francais et suppression des colonnes inutiles
df_title_akas_clean = df_title_akas.copy()
df_title_akas_clean = df_title_akas_clean.drop(columns=['ordering', 'language', 'types', 'attributes', 'isOriginalTitle'])

# Affiche les titres de la région FR
df_title_akas_clean = df_title_akas_clean[df_title_akas_clean['region'] == 'FR']

# Changement de la colonne 'title' en 'frenchTitle'
df_title_akas_clean = df_title_akas_clean.rename(columns={'title': 'frenchTitle'})

df_title_akas_clean.head(3)

,titleId,frenchTitle,region
12,tt0000002,Le clown et ses chiens,FR
21,tt0000003,Pauvre Pierrot,FR
30,tt0000004,Un bon bock,FR


In [11]:
# Jointure entre df_filtered et df_title_akas_clean, clé communune : tconst et titleId
df_filtered_french_title = pd.merge(df_filtered, df_title_akas_clean, left_on='tconst', right_on='titleId', how='left')

#  Suppression des colonnes titleId, region
df_filtered_french_title = df_filtered_french_title.drop(columns=['titleId', 'region'], axis=1)

# Si frenchTitle est NaN, on remplace par primaryTitle
df_filtered_french_title['frenchTitle'] = df_filtered_french_title['frenchTitle'].fillna(df_filtered_french_title['primaryTitle'])


# Enleve les '[]' et les ' dans genres_df_tmdb
df_filtered_french_title['genres_df_tmdb'] = df_filtered_french_title['genres_df_tmdb'].apply(lambda x: str(x).replace('[','').replace(']','').replace('\'', '').replace(' ',''))

# Enleve les '[]' et les ' dans production_countries
df_filtered_french_title['production_countries'] = df_filtered_french_title['production_countries'].apply(lambda x: str(x).replace('[','').replace(']','').replace('\'', '').replace(' ',''))

# Concatene les colonnes 'genres_df_title_basics' et 'genres_df_tmdb' en une seule colonne 'genres' et supprime les termes en doublon de la colonne 'genres'
df_filtered_french_title['genres'] = df_filtered_french_title.apply(lambda x: ', '.join(set(str(x['genres_df_title_basics']).split(',') + str(x['genres_df_tmdb']).split(','))), axis=1)    

# Supprime les colonnes 'genres_df_title_basics' et 'genres_df_tmdb'
df_filtered_french_title = df_filtered_french_title.drop(columns=['genres_df_title_basics', 'genres_df_tmdb'], axis=1)

# Range les colonnes
df_filtered_french_title = df_filtered_french_title[['tconst', 'primaryTitle',  'title', 'frenchTitle','startYear', 'genres', 'production_countries', 'runtimeMinutes', 'vote_average', 'vote_count', 'popularity', 'revenue', 'tagline', 'overview', 'id']]

df_filtered_french_title.head(3)

,tconst,primaryTitle,title,frenchTitle,startYear,genres,production_countries,runtimeMinutes,vote_average,vote_count,popularity,revenue,tagline,overview,id
0,tt0000591,The Prodigal Son,The Prodigal Son,L'enfant prodigue,1907,Drama,FR,90,0.0,0,0.600,0.0,NaN,The first feature-length motion picture produc...,396922
1,tt0001790,"Les Misérables, Part 1: Jean Valjean",Les Misérables - Part 1: Jean Valjean,Les misérables - Époque 1: Jean Valjean,1913,"Drama, History",FR,60,1.0,1,0.874,0.0,NaN,The story begins with Jean Valjean as a humble...,282219
2,tt0002375,La mort du duc d'Enghien,The Death of the Duke of Enghien,La mort du duc d'Enghien,1912,"Drama, History, Unknown",FR,<NA>,0.0,1,0.823,0.0,NaN,Directed by Albert Capellani.,285335


In [12]:
# Jointure entre df_filtered_french_title et df_title_ratings, clé commune : tconst
# On garde les colonnes 'tconst', 'averageRating', 'numVotes' et on supprime les colonnes 'vote_average', 'vote_count' et 'popularity' de df_filtered_french_title qui viennent de df_tmdb
df_filtered_ratings = pd.merge(df_filtered_french_title, df_title_ratings, left_on='tconst', right_on='tconst', how='left')

df_filtered_ratings = df_filtered_ratings.drop(columns=['vote_average', 'vote_count', 'popularity'], axis=1)
df_filtered_ratings['numVotes'] = df_filtered_ratings['numVotes'].astype('Int64')

# On enlève les films sans note
df_filtered_ratings = df_filtered_ratings[df_filtered_ratings['averageRating'].notna()]

# On enlève les films sortis avant 1990 dont la note est inférieure à 6 et le nombre de votes supérieurs a 1000, tout en  gardant les films sortis après 1990

df_filtered_ratings = df_filtered_ratings[~((df_filtered_ratings['startYear'].dt.year < 1990) & (df_filtered_ratings['averageRating'] < 6))]
df_filtered_ratings = df_filtered_ratings[~((df_filtered_ratings['startYear'].dt.year < 1990) & (df_filtered_ratings['numVotes'] < 1000))]

df_filtered_ratings.head(3)

,tconst,primaryTitle,title,frenchTitle,startYear,genres,production_countries,runtimeMinutes,revenue,tagline,overview,id,averageRating,numVotes
15,tt0006206,Les Vampires,"The Vampires or, The Arch Criminals of Paris",Les vampires,1915,"Adventure, Drama, Thriller, Crime, Action",FR,421,0.0,Every second exciting — every second thrilling...,Paris is prey to an invisible terror against w...,29082,7.3,5691
19,tt0006886,Judex,Judex,Judex,1916,"Adventure, Drama, Thriller, Crime, Mystery",FR,300,0.0,A serial in twelve episodes.,"When an unscrupulous banker ruins his family, ...",56801,7.2,1192
37,tt0010307,J'accuse!,J'accuse,J'accuse,1919,"Romance, Horror, Drama, History, War",FR,166,0.0,NaN,"The story of two men, one married, the other t...",70804,7.7,2241


In [13]:
# Suppression des doublons de la colonne 'tconst'
df_filtered_ratings_test = df_filtered_ratings.drop_duplicates(subset=['tconst'])
df_filtered_ratings_test.head(3)

,tconst,primaryTitle,title,frenchTitle,startYear,genres,production_countries,runtimeMinutes,revenue,tagline,overview,id,averageRating,numVotes
15,tt0006206,Les Vampires,"The Vampires or, The Arch Criminals of Paris",Les vampires,1915,"Adventure, Drama, Thriller, Crime, Action",FR,421,0.0,Every second exciting — every second thrilling...,Paris is prey to an invisible terror against w...,29082,7.3,5691
19,tt0006886,Judex,Judex,Judex,1916,"Adventure, Drama, Thriller, Crime, Mystery",FR,300,0.0,A serial in twelve episodes.,"When an unscrupulous banker ruins his family, ...",56801,7.2,1192
37,tt0010307,J'accuse!,J'accuse,J'accuse,1919,"Romance, Horror, Drama, History, War",FR,166,0.0,NaN,"The story of two men, one married, the other t...",70804,7.7,2241


In [14]:
# Merge des 3 df des acteurs et actrices/prodceurs etc...
df_merge_crew = pd.merge(df_title_crew, df_title_principals, left_on='tconst', right_on='tconst', how='left')
df_merge_crew_names = pd.merge(df_merge_crew, df_name_basics, left_on='nconst', right_on='nconst', how='left')

# On drop les colonnes inutiles -> ordering,  

df_merge_crew_names.head(3)


,tconst,directors,writers,ordering,nconst,category,job,characters,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,tt0000001,nm0005690,\N,1.0,nm1588970,self,\N,"[""Self""]",Carmencita,1868,1910,"soundtrack,archive_footage","tt0057728,tt0000001"
1,tt0000001,nm0005690,\N,2.0,nm0005690,director,\N,\N,William K.L. Dickson,1860,1935,"cinematographer,director,producer","tt1428455,tt1496763,tt0308254,tt0219560"
2,tt0000001,nm0005690,\N,3.0,nm0005690,producer,producer,\N,William K.L. Dickson,1860,1935,"cinematographer,director,producer","tt1428455,tt1496763,tt0308254,tt0219560"


In [15]:
df_merge_crew_names['category'].unique()

# On garde les categories les plus pertinentes pour suggerer un film a un néophyte -> actor, actress, director, producer, writer
df_merge_crew_names = df_merge_crew_names[df_merge_crew_names['category'].isin(['actor', 'actress', 'director', 'producer', 'writer'])]

# On suppime les colonnes inutiles -> ordering, directors, writers, nconst
df_merge_crew_names = df_merge_crew_names.drop(columns=['ordering', 'directors', 'writers', 'nconst'], axis=1)


In [16]:
df_merge_crew_names.head(3)

,tconst,category,job,characters,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
1,tt0000001,director,\N,\N,William K.L. Dickson,1860,1935,"cinematographer,director,producer","tt1428455,tt1496763,tt0308254,tt0219560"
2,tt0000001,producer,producer,\N,William K.L. Dickson,1860,1935,"cinematographer,director,producer","tt1428455,tt1496763,tt0308254,tt0219560"
4,tt0000002,director,\N,\N,Émile Reynaud,1844,1918,"director,animation_department,producer","tt0000003,tt32169968,tt16763740,tt16763674"


In [17]:
# Transformer chaque catégorie en une colonne 
df_sample = df_merge_crew_names.sample(1000000, random_state=42)


#df_merge_crew_names_col_cat = df_sample.pivot_table(index='tconst', columns='category', values='primaryName', aggfunc=list, fill_value=[]).reset_index()
#df_merge_crew_names_col_cat

In [18]:
"""df_temp = df_filtered_ratings_test[['tconst', 'primaryTitle']]


# Merge entre df_temp et df_merge_crew_names
df_final = pd.merge(df_temp, df_merge_crew_names, left_on='tconst', right_on='tconst', how='left')

df_final"""

"df_temp = df_filtered_ratings_test[['tconst', 'primaryTitle']]\n\n\n# Merge entre df_temp et df_merge_crew_names\ndf_final = pd.merge(df_temp, df_merge_crew_names, left_on='tconst', right_on='tconst', how='left')\n\ndf_final"

In [19]:
#df_sample = df_merge_crew_names.sample(1000000, random_state=42)

# Étape 1 : grouper et concaténer les noms pour chaque tconst + category
df_grouped = (
    df_merge_crew_names
      .groupby(['tconst', 'category'])['primaryName']
      .apply(lambda names: 
             ', '.join(
               sorted(
                 {n for n in names if pd.notnull(n) and isinstance(n, str)}
               )
             )
      )
      .reset_index()
)

# Étape 2 : pivot pour avoir une colonne par catégorie
df_pivot = df_grouped.pivot(index='tconst', columns='category', values='primaryName') \
    .reset_index()

# Étape 3 : nettoyage
df_pivot.columns.name = None  # enlève le nom de l’index de colonne

# Résultat
df_pivot.head(3)


,tconst,actor,actress,director,producer,writer
0,tt0000001,NaN,NaN,William K.L. Dickson,William K.L. Dickson,NaN
1,tt0000002,NaN,NaN,Émile Reynaud,NaN,NaN
2,tt0000003,NaN,NaN,Émile Reynaud,"Julien Pappé, Émile Reynaud",Émile Reynaud


In [20]:
# On enleve les colonnes category et primaryName
df_merge_crew_names_clean_cat = df_merge_crew_names.drop(columns=['category', 'primaryName'], axis=1)

# On concatene les colonnes 'birthYear' et 'deathYear' en une seule colonne 'birth_death_year'
df_merge_crew_names_clean_cat['deathYear'] = pd.to_numeric(df_merge_crew_names_clean_cat['deathYear'], errors='coerce')

df_merge_crew_names_clean_cat['birth_death_year'] = df_merge_crew_names_clean_cat.apply(lambda x: str(x['birthYear']) + '-' + str(x['deathYear']), axis=1)

# On supprime les colonnes birthYear et deathYear
df_merge_crew_names_clean_cat = df_merge_crew_names_clean_cat.drop(columns=['birthYear', 'deathYear', 'characters', 'primary'], axis=1)

# On merge df_merge_crew_names_clean_cat et df_pivot
df_merge_crew_final = pd.merge(df_merge_crew_names_clean_cat, df_pivot, left_on='tconst', right_on='tconst', how='left')

df_merge_crew_final.head(50)

KeyError: "['primary'] not found in axis"

In [ ]:
# Récuperer la liste des tconst dans df_filtered_ratings pour reduire la taille de df_merge_crew_final
df_tconst_crew = pd.merge(df_filtered_ratings_test, df_pivot, left_on='tconst', right_on='tconst', how='left')

# Créer un filtre pour la colonne runtimeMinutes qui n'affiche que les films de plus de 60 minutes et de moins de 250 minutes
df_tconst_crew = df_tconst_crew[(df_tconst_crew['runtimeMinutes'] > 60) & (df_tconst_crew['runtimeMinutes'] < 250)]

df_tconst_crew.head(3)

,tconst,primaryTitle,title,frenchTitle,startYear,genres,production_countries,runtimeMinutes,revenue,tagline,overview,id,averageRating,numVotes,actor,actress,director,producer,writer
2,tt0010307,J'accuse!,J'accuse,J'accuse,1919,"Horror, War, Romance, Drama, History",FR,166,0.0,NaN,"The story of two men, one married, the other t...",70804,7.7,2240,"Blaise Cendrars, Maxime Desjardins, Paul Duc, ...","Angèle Guys, Elizabeth Nizan, Mancini, Maryse ...",Abel Gance,Charles Pathé,Abel Gance
3,tt0013933,The Faithful Heart,Cœur fidèle,Coeur fidèle,1923,"Romance, Drama",FR,87,0.0,NaN,The good guys win out in this sweet tale about...,67440,7.4,1540,"Claude Benedict, Edmond Van Daële, Léon Mathot","Gina Manès, Madame Maufroy, Madeleine Erickson...",Jean Epstein,NaN,"Jean Epstein, Marie Epstein"
5,tt0015014,L'inhumaine,L'Inhumaine,L'inhumaine,1924,"Romance, ScienceFiction, Drama, Mystery",FR,135,0.0,NaN,"A famous singer Claire Lescot, who lives on th...",54539,7.2,1134,"Fred Kellerman, Jaque Catelain, Las Bonambella...","Georgette Leblanc, Kiki of Montparnasse, Marce...",Marcel L'Herbier,Marie-Laure de Noailles,"Georgette Leblanc, Joris-Karl Huysmans, Marcel..."


In [ ]:
df_tconst_crew.describe()


,runtimeMinutes,revenue,id,averageRating,numVotes
count,12665.0,1.266500e+04,12665.0,12665.000000,12665.0
mean,100.866403,7.959074e+06,270840.243111,6.068006,17891.515989
std,18.336841,5.654873e+07,288564.234711,1.132547,86636.947512
min,61.0,0.000000e+00,16.0,1.300000,6.0
25%,90.0,0.000000e+00,41271.0,5.500000,315.0
50%,98.0,0.000000e+00,128593.0,6.300000,1307.0
75%,108.0,0.000000e+00,450050.0,6.800000,5482.0
max,240.0,2.923706e+09,1121899.0,9.700000,3019825.0


In [ ]:
df_tconst_crew.to_csv('df_final.csv', index=False, sep=';')

In [ ]:
df_final = pd.read_csv('df_final.csv', sep=';', low_memory=False)
df_final

,tconst,primaryTitle,title,frenchTitle,startYear,genres,production_countries,runtimeMinutes,revenue,tagline,overview,id,averageRating,numVotes,actor,actress,director,producer,writer
0,tt0010307,J'accuse!,J'accuse,J'accuse,1919,"Horror, War, Romance, Drama, History",FR,166,0.0,NaN,"The story of two men, one married, the other t...",70804,7.7,2240,"Blaise Cendrars, Maxime Desjardins, Paul Duc, ...","Angèle Guys, Elizabeth Nizan, Mancini, Maryse ...",Abel Gance,Charles Pathé,Abel Gance
1,tt0013933,The Faithful Heart,Cœur fidèle,Coeur fidèle,1923,"Romance, Drama",FR,87,0.0,NaN,The good guys win out in this sweet tale about...,67440,7.4,1540,"Claude Benedict, Edmond Van Daële, Léon Mathot","Gina Manès, Madame Maufroy, Madeleine Erickson...",Jean Epstein,NaN,"Jean Epstein, Marie Epstein"
2,tt0015014,L'inhumaine,L'Inhumaine,L'inhumaine,1924,"Romance, ScienceFiction, Drama, Mystery",FR,135,0.0,NaN,"A famous singer Claire Lescot, who lives on th...",54539,7.2,1134,"Fred Kellerman, Jaque Catelain, Las Bonambella...","Georgette Leblanc, Kiki of Montparnasse, Marce...",Marcel L'Herbier,Marie-Laure de Noailles,"Georgette Leblanc, Joris-Karl Huysmans, Marcel..."
3,tt0017075,The Lodger: A Story of the London Fog,The Lodger: A Story of the London Fog,Les cheveux d'or,1927,"Thriller, Drama, Crime, Mystery",GB,92,0.0,NaN,London. A mysterious serial killer brutally mu...,2760,7.3,14289,"Arthur Chesney, Ivor Novello, Malcolm Keen, Re...","Daisy Campbell, Eve Gray, June Tripp, Marie Au...",Alfred Hitchcock,NaN,"Eliot Stannard, Marie Belloc Lowndes"
4,tt0017196,Nana,Nana,Nana,1926,"Romance, Drama",FR,150,0.0,NaN,"A government official, Count Muffat, falls und...",66812,6.6,1011,"Claude Autant-Lara, Jean Angelo, Karl Harbache...","Catherine Hessling, Jacqueline Forzane, Valesk...",Jean Renoir,Jean Renoir,"Denise Leblond, Pierre Lestringuez, Émile Zola"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12660,tt9902160,Herself,Herself,Herself,2020,Drama,"IE,GB",97,0.0,Life under construction,Struggling to provide her daughters with a saf...,653708,7.0,5099,"Art Kearns, Ian Lloyd Anderson, Shadaan Felfeli","Anita Petry, Cathy Belton, Clare Dunne, Ericka...",Phyllida Lloyd,"Ed Guiney, Rory Gilmartin, Sharon Horgan","Clare Dunne, Malcolm Campbell"
12661,tt9904802,Enemy Lines,Enemy Lines,Enemy Lines,2020,"Drama, Action, War",GB,92,0.0,Courage has no border.,"In the frozen, war torn landscape of occupied ...",679796,4.6,2041,"Corey Johnson, Daniel Jillings, Ed Westwick, G...",Maria Bondareva,Anders Banke,"Aleksandr Kushaev, Andy Thompson, Nadzeya Huse...","Michael Wright, Tom George"
12662,tt9908390,Le lion,The Lion,Le lion,2020,Comedy,FR,95,0.0,NaN,A psychiatric hospital patient pretends to be ...,589970,5.5,1497,"Aksel Ustun, Benoît Pétré, Dany Boon, Mathieu ...","Anne Serra, Carole Brana, Sophie Verbeeck",Ludovic Colbeau-Justin,"Jean-Yves Robin, Marc Stanimirovic, Serge Hayat","Alexandre Coquelle, Matthieu Le Naour"
12663,tt9916190,Safeguard,Safeguard,Safeguard,2020,"Thriller, Adventure, Action, Crime",GB,95,0.0,Protecting your family comes at a cost.,When a Japanese restaurant is extorted by the ...,633490,3.6,263,"Akie Kotabe, Andrei Nova, Lee Byford, Leo Ashi...",Yuriri Naka,Fraser Precious,"Fraser Precious, Megan Young",Fraser Precious
